In [ ]:
# ID rough cost for doing semantic chunking

In [17]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
import tiktoken
import os
from dotenv import load_dotenv

# Load OpenAI API key from .env file
load_dotenv("../../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Read in files (average size and largest)

In [23]:
with open("../data/dss-non-medicaid-pt2.txt") as f:
    largest_transcript = f.read()
with open("../data/ltgov-dhss-p1.txt") as f:
    average_transcript = f.read()
transcripts = {
    'largest_transcript': largest_transcript,
    'average_transcript': average_transcript,
}

In [15]:
# Instantiate Text Splitter

In [14]:
text_splitter = SemanticChunker(OpenAIEmbeddings(openai_api_key=openai_api_key))

In [29]:
## Estimate text splitting cost for documents
# translate costs to per token
embedding_cost = {
    'text-embedding-3-small' = .02/1000000,
    'text-embedding-3-large' = .13/1000000
}

for transcript in transcripts:
    print(transcript)
    for model in ['text-embedding-3-small', 'text-embedding-3-large']:
        encoding = tiktoken.encoding_for_model(model)
        num_tokens = len(encoding.encode(transcripts[transcript]))
        print(f"* {model}")
        print(f"** num of tokens: {num_tokens}")
        print(f"** estimated cost: {embedding_cost[model] * num_tokens}"

SyntaxError: cannot assign to literal here. Maybe you meant '==' instead of '='? (2485561463.py, line 4)